# DTree Example

In [1]:
from DTree import DTree
from DTree.Formula import formula

from Lifting import Lifting
import json
import time

In [2]:
lifting = Lifting.Lifting()
with open("data/hard/factorisedFormulas/17e_n_name4_factorised.json") as fp:
    JsonObject = json.load(fp)
lineage = JsonObject[0]  
testFormula = formula(lineage)
original_variables = testFormula.get_variable_dict(testFormula.subformula).keys()
current_id, lifting_dict = lifting.lift_read_once_clause(testFormula)
lifted_variable_dict, global_var_to_id_dict_test = lifting.lift_formula(testFormula, current_id)
lifting_dict.update(lifted_variable_dict)

In [3]:
dtree = DTree.Dtree(testFormula, None)
print(dtree)

DTREE_GATE.Independent_And DTREE_GATE.Independent_Or DTREE_GATE.Independent_Or DTREE_GATE.Independent_Or DTREE_GATE.Exclusive_Or DTREE_GATE.Independent_Or {"operator": and,
 "subformula": [16, 9, 10, 11, 12, 13, 14, 15]} 81 DTREE_GATE.Exclusive_Or DTREE_GATE.Exclusive_Or DTREE_GATE.Exclusive_Or DTREE_GATE.Exclusive_Or DTREE_GATE.Exclusive_Or DTREE_GATE.Exclusive_Or DTREE_GATE.Exclusive_Or True False False False False False False False DTREE_GATE.Independent_Or DTREE_GATE.Exclusive_Or DTREE_GATE.Independent_Or 269 {"operator": or,
 "subformula": [80, 202, 162]} DTREE_GATE.Exclusive_Or DTREE_GATE.Independent_Or 80 162 DTREE_GATE.Exclusive_Or DTREE_GATE.Exclusive_Or True False False DTREE_GATE.Independent_Or {"operator": and,
 "subformula": [09a9ff5d-43be-4715-8287-3102cb666739, 78]} DTREE_GATE.Independent_Or {"operator": and,
 "subformula": [af254d6a-f68a-42f3-baf2-7b26acbc3dee, 203]} {"operator": or,
 "subformula": [77, 79, 82, 83, 84, 85, 200, 201, 204, 205, 206, 207, 208, 209, 154, 15

In [4]:
def calculate_banzhaf(dtree, original_variables, lifting_dict, timeout=3600):
    if dtree.variable_count == 0:
        return {"status": "Success", "expansion_time": 0, "calculation_time": 0, "depth": 0, "banzhaf_values": {}}
    banzhaf_vals = {}
    var_to_similar_vars = {}
    var_to_lifted_var = {}

    for element in lifting_dict.items():
        for var in element[1]["subformula"]:
            var_to_lifted_var[var] = element[0]
            var_to_similar_vars[var] = element[1]["subformula"].copy()
            var_to_similar_vars[var].remove(var)
    
    for var in original_variables:
        if var not in var_to_similar_vars:
            var_to_similar_vars[var] = []

    start = time.time()

    for fact in original_variables:
        if fact not in banzhaf_vals:
            result = dtree.critical_assignments_fact(fact, var_to_lifted_var.copy(), lifting_dict.copy())
            banzhaf_vals[fact] = result
            for var in var_to_similar_vars[fact]:
                if var not in banzhaf_vals:
                    banzhaf_vals[var] = result
        mid = time.time()
        if mid - start > timeout:
            return {"status": "Failed", "calculation_time": "Failed", "banzhaf_values": banzhaf_vals}
    end = time.time()
    calc_time = end - start
    return {"status": "Success","calculation_time": calc_time, "banzhaf_values": banzhaf_vals}

In [5]:
original_variables = list(original_variables)
result = calculate_banzhaf(dtree, original_variables, lifting_dict, 3600)
print(result)

{'status': 'Success', 'calculation_time': 0.5259943008422852, 'banzhaf_values': {'c9e895c0-9723-483d-9ffb-7f711c193e6a': 643668733004715271882624132097006887877308574623396604346091905348911633116645770056864375892847985475754826670180029200874323000082596164003403119289720393652538239807268352585668, '86cb7f93-4714-4c2e-9db3-e244fee6097d': 643668733004715271882624132097006887877308574623396604346091905348911633116645770056864375892847985475754826670180029200874323000082596164003403119289720393652538239807268352585668, '063c8549-f56a-4689-bc21-a2262c2ee70a': 0, '2cffb2c8-f0f4-44e7-b747-7aa212bc2920': 0, '0bc4d825-b280-4b5b-8b1e-57d4fd7d019d': 0, '129453b5-3cf6-4398-95a6-62aae5e72766': 0, '344ea546-9f57-48b5-9ecc-f2b8c799fd8e': 0, '170ca683-a48c-473f-a63f-9f7a2af6f7a7': 0, 'fb44129b-5ad0-4399-99e8-61d3dc375dec': 0, '1b5dd1f1-8def-44c3-8334-426180d05cb3': 0, 'af254d6a-f68a-42f3-baf2-7b26acbc3dee': 0, '23a3bb12-e475-4470-bcb3-61eea665c259': 0, '988a7eff-b495-42c4-b805-b339a09c17e4': 0, '2